In [6]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [31]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [32]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
    
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(maxlen)
        positions = self.pos_emb(positions)
        tokens = self.token_emb(x)
        return tokens + positions

In [33]:
vocab_size = 20000
maxlen = 200
(x_train, y_train),(x_val,y_val) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
x_train = tf.keras.utils.pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post',)
x_val = tf.keras.utils.pad_sequences(x_val, maxlen=maxlen, padding='post', truncating='post',)

In [44]:
embed_dim = 32
num_heads = 2
ff_dim = 32
inputs = tf.keras.layers.Input(shape=(maxlen,))
x = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)(inputs)
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss = tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics = [["accuracy"]])
history = model.fit(x_train, y_train, batch_size=32, epochs=3 ,validation_data=(x_val, y_val))

Epoch 1/3
782/782 [==============================] - 4s 5ms/step - loss: 0.4113 - accuracy: 0.8008 - val_loss: 0.3219 - val_accuracy: 0.8617
Epoch 2/3
782/782 [==============================] - 3s 4ms/step - loss: 0.2196 - accuracy: 0.9168 - val_loss: 0.3474 - val_accuracy: 0.8518
Epoch 3/3
782/782 [==============================] - 3s 4ms/step - loss: 0.1427 - accuracy: 0.9488 - val_loss: 0.4275 - val_accuracy: 0.8384
